In [95]:
!sudo apt install tesseract-ocr

!pip install pytesseract

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.00~git2288-10f4998a-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [96]:
pip install img2pdf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [97]:
!pip install tesseract

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [98]:
pip install opencv-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [99]:
pip install flask


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [100]:
!pip install tabula-py


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [101]:
pip install tabula

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [102]:
!apt-get install ghostscript

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ghostscript is already the newest version (9.26~dfsg+0-0ubuntu0.18.04.16).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.


In [103]:
pip install ocrmypdf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [127]:
import sys
import numpy as np
from cv2 import *
import time
import os
import threading
import pytesseract
import imutils
import cv2
from PIL import Image
from tabula.io import read_pdf
import tabula
import pandas as pd
import ocrmypdf

In [151]:
    img_path=r'/content/FER 27-28 57-58_pages-to-jpg-0003.jpg'
    # Read image using opencv
    file_name = os.path.basename(img_path)
    # file name without extension
    filename =os.path.splitext(file_name)[0]
    img = cv2.imread(img_path)
   # Extract the file name without the file extension
    file_name = os.path.splitext(img_path)[0]
    #file_name = file_name
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #Removing Shadows
    rgb_planes = cv2.split(img)
    result_planes = []
    result_norm_planes = []
    for plane in rgb_planes:
        dilated_img = cv2.dilate(plane, np.ones((7,7), np.uint8))
        bg_img = cv2.medianBlur(dilated_img, 21)
        diff_img = 255 - cv2.absdiff(plane, bg_img)
        result_planes.append(diff_img)
    img = cv2.merge(result_planes)
    
    #Apply dilation and erosion to remove some noise
    kernel = np.ones((1, 1), np.uint8)
    img = cv2.dilate(img, kernel, iterations=1)#increases the white region in the image 
    img = cv2.erode(img, kernel, iterations=1) #erodes away the boundaries of foreground object
    
    # Apply threshold to get image with only b&w (binarization)
    img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    img = cv2.imwrite('/content/1.png',img)
    

In [152]:
#image to pdf
img = Image.open('/content/1.png')
img_pfd = img.save(r'1.pdf')


In [153]:
#image pdf to text pdf
def scannedPdfConverter(file_path, save_path):
    ocrmypdf.ocr(file_path, save_path, skip_text=True)
    print('File converted successfully!')

file_path = "/content/1.pdf"
save_path ="/content/11.pdf"
scannedPdfConverter(file_path, save_path)

Scanning contents: 100%|██████████| 1/1 [00:00<00:00, 154.73page/s]
OCR: 100%|██████████| 1.0/1.0 [00:02<00:00,  2.27s/page]
PDF/A conversion: 100%|██████████| 1/1 [00:00<00:00,  6.84page/s]
Recompressing JPEGs: 0image [00:00, ?image/s]
Deflating JPEGs: 100%|██████████| 1/1 [00:00<00:00, 87.35image/s]
JBIG2: 0item [00:00, ?item/s]

File converted successfully!


In [154]:
# text pdf to csv file 
df = tabula.read_pdf("/content/11.pdf", pages='all')[0]
df = pd.DataFrame(df)
print(df)
df.to_csv("11.csv", index=False)

   Depth         GR      CALI         R,      Veu PIGN(®,}       Sw
0      m       BAPI        in      ohm.m   m3/m3.    m3/m3    m3/m3
1   3476    71.7252   8.71369   30.88494  0.12694  0.14111  0.39672
2   3477   64.38234   8.97041   27.48025  0.10324  0.15164  0.41772
3   3478   69.16407   8.92308    42.8899  0.10108  0.12605  0.40134
4   3479   68.87645   8.40721    19.9842  0.10963  0.12709  0.57149
5   3480   72.29375   8.52481   28.73718  0.11358  0.11002   0.3324
6   3481   58.89062   8.30769   31.85952  0.11153   0.2012  0.31013
7   3482   45.38615    8.4456  190,25443  0.04145  0.07491  0,30841
8   3483   59.24623   8.40237   45,79617  0.13055  0.10917  0.45814
9   3484   61.69956   8.61538   74.13807  0.10209  0.07096  0.29466
10  3485   57.91767   8.48521   25.25423   0.0953  0.14534  0.45478
11  3486   76.57812   8.71472   26.62168  0.14166  0.13261  0.44492
12  3487   85.72929   8.40237   23.69687  0.16059  0.07408  0.49364
13  3488  101.75997   8.59172     23.758  0.2248

In [155]:
#append the file name of the original image 
filename_suffix = 'csv'
new_path = os.path.join('/content/', filename + '.' + filename_suffix)
source= '/content/11.csv'
os.rename(source, new_path)

In [156]:
#removing unnecessary files 
os.remove("/content/1.pdf")
os.remove("/content/1.png")
os.remove("/content/11.pdf")
